## Art- Database Continued - Descriptions

In [267]:
# import libraries

import pandas as pd
import numpy as np
import requests
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import re
from transformers import pipeline
from IPython.display import Image, display
import math

# set options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows', 100)
import warnings 
  

warnings.filterwarnings('ignore') 

In [269]:
#load to dataframe
df = pd.read_csv('objects.csv')

dropped_cols = ['contextualtextcount', 'accesslevel', 'dateoflastpageview', 'totaluniquepageviews', 'markscount',
                'publicationcount', 'totaluniquepageviews', 'contact', 'colorcount', 'rank', 'id', 'state', 
                'verificationleveldescription', 'images','imagecount', 'worktypes', 'imagecount', 'classificationid',
                'standardreferencenumber', 'signed', 'relatedcount', 'verificationlevel', 'titlescount',
                'peoplecount', 'lastupdate', 'periodid', 'edition', 'lendingpermissionlevel', 'colors', 'groupcount',
                'dateoffirstpageview', 'datebegin', 'exhibitioncount', 'imagepermissionlevel', 'mediacount', 'techniqueid', 
                'seeAlso', 'details', 'videos', 'audio']

df.drop(dropped_cols, axis = 1, inplace = True)

## Grab just the records with descriptions

In [271]:
description_df = df[~df['description'].isna()]
print(len(description_df))
display(description_df[['description']][:100])

36688


,description
0,Corresponds to negative BRLF.457.10
1,Corresponds to negative BRLF.196.13
2,Corresponds to negative BRLF.413.6
4,Corresponds to negative BRLF.106.2
6,Corresponds to negative BRLF.106.9
9,"illustrates April, corresponding zodiac sign (cancer) at top center"
10,"illustrates December, corresponding zodiac sign (capricorn) at the center top"
12,"The postcard bears the printed signature ""E.Hansen""\r\nPrinted at l.l.: F. Killinger, Zürich. Nr. 27.\r\n"
13,"This painting depicts figures marching into battle on chariots and horse-backs, drawing their weapons towards their enemy. This painting is part of a four-folio scene, which, combined, would create a large battle painting. It is possible that this folio, along with the other three, was made sometime after the Dispersed Bhagavata Purana series was completed to accompany the series. There is no text on the reverse of this folio. \r\n\r\nThis painting belongs to the 'Dispersed' Bhagavata Purana series, produced in the early 16th century, possibly in Mathura. The series, possibly consisting of 360 such folios, is based on the tenth book of the Bhagavata Purana, which describes the story of Krishna, particularly his childhood and youth. This series was one of South Asia's earliest illustrated Bhagavata Purana manuscripts. Other folios from the same Bhagavata Purana series in the Harvard Art Museum’s collection are objects 1963.145, 1974.124, 1974.125, 1974.126, 1974.127, 1992.283, 1995.67, 1995.68. Caurapancasika-group style.\r\n"
14,"This handscroll depicts a range of tree-covered mountains shrouded in mist. A title written by the artist precedes the painting; the title is a line from a 1936 poem by Mao Zedong and reads ""Jiangshan ruci dou jiao"". It can be translated as ""What a Glorious Land!"", though it is also translated as ""This land, so rich in beauty."""


## There are 36688 records with descriptions. Now lets remove records with formated descriptions


In [273]:
description_df =description_df[~description_df.description.str.contains('|'.join(['Corresponds to negative',
                        'Corresponds with negative', "Postcard", 'Corresponds to slide', 'in paper portfolio cover'  
                          'Small-format print', 'Two photographs printed on a single sheet',  'Obv.: aegis\r\nRev.:'                         ]),case=False )]
print(len(description_df))
display(description_df[['description']][:100])

35889


,description
9,"illustrates April, corresponding zodiac sign (cancer) at top center"
10,"illustrates December, corresponding zodiac sign (capricorn) at the center top"
13,"This painting depicts figures marching into battle on chariots and horse-backs, drawing their weapons towards their enemy. This painting is part of a four-folio scene, which, combined, would create a large battle painting. It is possible that this folio, along with the other three, was made sometime after the Dispersed Bhagavata Purana series was completed to accompany the series. There is no text on the reverse of this folio. \r\n\r\nThis painting belongs to the 'Dispersed' Bhagavata Purana series, produced in the early 16th century, possibly in Mathura. The series, possibly consisting of 360 such folios, is based on the tenth book of the Bhagavata Purana, which describes the story of Krishna, particularly his childhood and youth. This series was one of South Asia's earliest illustrated Bhagavata Purana manuscripts. Other folios from the same Bhagavata Purana series in the Harvard Art Museum’s collection are objects 1963.145, 1974.124, 1974.125, 1974.126, 1974.127, 1992.283, 1995.67, 1995.68. Caurapancasika-group style.\r\n"
14,"This handscroll depicts a range of tree-covered mountains shrouded in mist. A title written by the artist precedes the painting; the title is a line from a 1936 poem by Mao Zedong and reads ""Jiangshan ruci dou jiao"". It can be translated as ""What a Glorious Land!"", though it is also translated as ""This land, so rich in beauty."""
16,"Double-sided card printed with the statement ""'Ben's statement on the other side of this card is true' George Brecht"" on one side, and ""'George Brecht's statement on the other side of this card is false' Ben"" on the other. Included in Flux Year Box 2."
46,"A miniature version of Brecht's ""Games & Puzzles"" box editioned as part of Flux Year Box 2. It contains instructions for ""Swim Puzzle"", ""Bead Puzzle"", and ""Ball Puzzle""."
50,"A ""Yamfest Movie"" in the form of a flipbook. Included by Maciunas in ""Flux Year Box 2""."
54,"illustrates June, corresponding sign of zodiac (cancer) at top center"
55,"illustrates August, corresponding zodiac sign (virgo) at top center"
56,"illustrates September, corresponding zodiac sign (libra) at top center"


## Simple Sentiment Analysis
### try on description field

In [276]:
# Initialize the sentiment analysis pipeline
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

def clean_text(text):
    """Preprocess text by removing leading numbers/special characters"""
    if pd.isna(text) or str(text).strip() == "":
        return ""
    
    text = str(text)
    # Remove leading numbers followed by punctuation (like "24) ")
    text = re.sub(r'^\d+[\)\.,]\s*', '', text)
    # Remove any other leading special characters
    text = re.sub(r'^[^a-zA-Z0-9"\']+', '', text)
    return text.strip()

def get_sentiment(text):
    try:
        cleaned_text = clean_text(text)
        if not cleaned_text:  # Skip empty text after cleaning
            return None
        
        # Truncate very long texts to model's max length (512 tokens)
        max_length = 500  # Leaving some room for tokenization
        if len(cleaned_text) > max_length:
            cleaned_text = cleaned_text[:max_length]
        
        result = sentiment_pipeline(cleaned_text)[0]
        return result['label']
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return None

# Apply to DataFrame
#description_df['desc_sentiment'] = description_df['description'].apply(get_sentiment)

# For sentiment with score
def get_sentiment_with_score(text):
    try:
        cleaned_text = clean_text(text)
        if not cleaned_text:
            return None, None
        
        max_length = 500
        if len(cleaned_text) > max_length:
            cleaned_text = cleaned_text[:max_length]
        
        result = sentiment_pipeline(cleaned_text)[0]
        return result['label'], result['score']
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return None, None

description_df[['desc_sentiment', 'sentiment_score']] = description_df['description'].apply(
    lambda x: pd.Series(get_sentiment_with_score(x))
)



Device set to use mps:0


In [210]:
# Display results
display(description_df[['description', 'desc_sentiment', 'sentiment_score']][:50])

,description,desc_sentiment,sentiment_score
9,"illustrates April, corresponding zodiac sign (cancer) at top center",NEGATIVE,0.958537
10,"illustrates December, corresponding zodiac sign (capricorn) at the center top",NEGATIVE,0.589319
13,"This painting depicts figures marching into battle on chariots and horse-backs, drawing their weapons towards their enemy. This painting is part of a four-folio scene, which, combined, would create a large battle painting. It is possible that this folio, along with the other three, was made sometime after the Dispersed Bhagavata Purana series was completed to accompany the series. There is no text on the reverse of this folio. \r\n\r\nThis painting belongs to the 'Dispersed' Bhagavata Purana series, produced in the early 16th century, possibly in Mathura. The series, possibly consisting of 360 such folios, is based on the tenth book of the Bhagavata Purana, which describes the story of Krishna, particularly his childhood and youth. This series was one of South Asia's earliest illustrated Bhagavata Purana manuscripts. Other folios from the same Bhagavata Purana series in the Harvard Art Museum’s collection are objects 1963.145, 1974.124, 1974.125, 1974.126, 1974.127, 1992.283, 1995.67, 1995.68. Caurapancasika-group style.\r\n",NEGATIVE,0.997807
14,"This handscroll depicts a range of tree-covered mountains shrouded in mist. A title written by the artist precedes the painting; the title is a line from a 1936 poem by Mao Zedong and reads ""Jiangshan ruci dou jiao"". It can be translated as ""What a Glorious Land!"", though it is also translated as ""This land, so rich in beauty.""",POSITIVE,0.998657
16,"Double-sided card printed with the statement ""'Ben's statement on the other side of this card is true' George Brecht"" on one side, and ""'George Brecht's statement on the other side of this card is false' Ben"" on the other. Included in Flux Year Box 2.",NEGATIVE,0.994584
46,"A miniature version of Brecht's ""Games & Puzzles"" box editioned as part of Flux Year Box 2. It contains instructions for ""Swim Puzzle"", ""Bead Puzzle"", and ""Ball Puzzle"".",NEGATIVE,0.959846
50,"A ""Yamfest Movie"" in the form of a flipbook. Included by Maciunas in ""Flux Year Box 2"".",NEGATIVE,0.995656
54,"illustrates June, corresponding sign of zodiac (cancer) at top center",NEGATIVE,0.884190
55,"illustrates August, corresponding zodiac sign (virgo) at top center",POSITIVE,0.956830
56,"illustrates September, corresponding zodiac sign (libra) at top center",POSITIVE,0.978172


## Commentary Analysis
## Grab just the records with commentary, there are 2424 of them

In [278]:
commentary_df = df[~df['commentary'].isna()]
print(len(commentary_df))
display(commentary_df[['commentary']][:100])

2424


,commentary
148,"The patka as an item of clothing has a long history but found its most elegant expression in the age of the Mughals. Examples of it can be seen in almost every portrait of Mughal emperors and courtiers from the late 16th century to the 19th century. The patka was a long, narrow, textile associated with courtly dress. The patka was worn about the wearer's waist, over top a long shirt."
154,"Index 15 October 1953. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]."
174,"Index 11 May 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil (mural) Composition Concrete, 1957 (Davis Catalogue Raisonné‚ no. 1701)."
175,"Index 27 October 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
176,"Index 27 October 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
177,"Index 7 November 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
178,"Index 7 November 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
179,"Index 15 December 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
180,"Index 15 December 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Pochade, 1956-58 (Davis Catalogue Raisonné‚ no. 1711)."
187,"Index 7 November 1957. For index reference, see Finding Aid [nrs.harvard.edu/urn-3:HUAM:art00006]. See related oil Standard Still Life, 1958 (Davis Catalogue Raisonné‚ no. 1706)."


## Remove entries that start with 'Index", this leaves 1258 records

In [280]:
commentary_df = commentary_df[~commentary_df.commentary.str.contains('Index')]
print(len(commentary_df))
display(commentary_df[['commentary']][:100])
commentary_df.to_csv('comments.csv')

1258


,commentary
148,"The patka as an item of clothing has a long history but found its most elegant expression in the age of the Mughals. Examples of it can be seen in almost every portrait of Mughal emperors and courtiers from the late 16th century to the 19th century. The patka was a long, narrow, textile associated with courtly dress. The patka was worn about the wearer's waist, over top a long shirt."
270,"The use of these types of disks, sometimes called ""dischi sacri"" (""sacred disks""), is not entirely understood. Several are known, found in Italy, particularly in the south and around ancient Taras/Tarentum (Taranto). Some are in relief, but most have the symbols impressed into the disk.\r\n\r\nThe disks often bear many of the same motifs, some arranged in similar configurations from object to object. The objects may represent divinities--some common associations like Zeus with the thunderbolt, Hephaistos with the tongs, Poseidon with the trident, Hermes with the caduceus, Apollo with the lyre, and Athena with the owl are very clear--but many are in fact everyday items that are not necessrily associated with any particular god or goddess. They may, therefore, represent a mixture of sacred and profane objects, a selection of offerings, or as Prof. McDaniel suggested in his 1924 article about these disks, items loosely associated with the city of Tarentum. Favoring the latter explanation, McDaniel suggested that the symbols formed a ""compound seal"" used in commerce, associated with Tarentum. They would have been impressed in wax or on clay to form seal impressions that would be particularly difficult to duplicate due to their complexity."
319,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of the prints, Trio, Devoted, and Movement in White convey the artist's life-long interest in spiritual themes."
441,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of the prints, Trio, Devoted, and Movement in White convey the artist's life-long interest in spiritual themes."
469,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of the prints, Trio, Devoted, and Movement in White convey the artist's life-long interest in spir

## With only 1258 records we could likely read them all, lets try sentiment analysis to see if we are able to pick anything up

In [216]:
commentary_df[['com_sentiment', 'com_score']] = commentary_df['commentary'].apply(
    lambda x: pd.Series(get_sentiment_with_score(x))
)

In [217]:
# Display results
display(commentary_df[['commentary', 'com_sentiment', 'com_score']][:100])

,commentary,com_sentiment,com_score
148,"The patka as an item of clothing has a long history but found its most elegant expression in the age of the Mughals. Examples of it can be seen in almost every portrait of Mughal emperors and courtiers from the late 16th century to the 19th century. The patka was a long, narrow, textile associated with courtly dress. The patka was worn about the wearer's waist, over top a long shirt.",POSITIVE,0.629711
270,"The use of these types of disks, sometimes called ""dischi sacri"" (""sacred disks""), is not entirely understood. Several are known, found in Italy, particularly in the south and around ancient Taras/Tarentum (Taranto). Some are in relief, but most have the symbols impressed into the disk.\r\n\r\nThe disks often bear many of the same motifs, some arranged in similar configurations from object to object. The objects may represent divinities--some common associations like Zeus with the thunderbolt, Hephaistos with the tongs, Poseidon with the trident, Hermes with the caduceus, Apollo with the lyre, and Athena with the owl are very clear--but many are in fact everyday items that are not necessrily associated with any particular god or goddess. They may, therefore, represent a mixture of sacred and profane objects, a selection of offerings, or as Prof. McDaniel suggested in his 1924 article about these disks, items loosely associated with the city of Tarentum. Favoring the latter explanation, McDaniel suggested that the symbols formed a ""compound seal"" used in commerce, associated with Tarentum. They would have been impressed in wax or on clay to form seal impressions that would be particularly difficult to duplicate due to their complexity.",NEGATIVE,0.986743
319,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of the prints, Trio, Devoted, and Movement in White convey the artist's life-long interest in spiritual themes.",POSITIVE,0.991832
441,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of the prints, Trio, Devoted, and Movement in White convey the artist's life-long interest in spiritual themes.",POSITIVE,0.991832
469,"These three prints are from Mark Tobey's Transitions portfolio of seven aquatints. Tobey, one of the ""mystical painters of the Northwest,"" had become a member of the Bahai World Faith in 1918 and in 1934 visited China and Japan. His noted ""white writing"" style of painting--an overlay of white calligraphic style symbols on an abstract field--evolved from his study of Asian calligraphy and brush painting during this visit. Tobey spent much of his life working in the Seattle area but in 1960 moved to Basel, Switzerland. The Transitions portfolio was published in Frankfurt in 1970 and the prints clearly grow out of his ""white writing"" style. The titles of

In [218]:
display(commentary_df.loc[[5993, 24419, 24934, 11545]][['title', 'commentary', 'com_sentiment', 'com_score']])

,title,commentary,com_sentiment,com_score
5993,Untitled (Portfolio of Twelve Prints),"As in much of his other work, Richard Prince explores issues of masculinity in this portfolio of twelve lithographs. Each print comprises a smattering of texts and images: some fragmentary, some fully realized, some drawn on the stone, some transferred from other sources. Prince's hand-written texts tell jokes, the kinds of raunchy, yet classic jokes allegedly told among men. The prints are an extension of his painting practice. In the late 1980s, Prince began a series of joke paintings that critics have related to the humor and cartoon imagery featured in men's magazines like Playboy. In fact, the portfolio box that contains the prints the prints is decorated with a Playboy bunny on its cover. Yet although Prince deploys such iconic signifiers of masculinity as the bunny and dirty traveling salesmen jokes, the picture of masculinity he conjures is a confusing questioning one. In fact even the Playboy bunny is manipulated-it's given a dangerous-looking set of teeth. Included among the texts are affirmations of male homosexuality, certainly a subject taboo in men's porn magazines. The print publisher Susan Inglett claims that the drawings for the prints were made after prince and his wife separated and he was engaged in a homosexual affair. Whatever his personal stake in the imagery is, the prints examine masculinity in a sensitive, smart, and unusual way.",POSITIVE,0.960970
24419,Portfolio,"This lithograph is very much a latter-day Tanning, and she herself admits it is a retrospective self-portrait, according to the dealer from which this was purchased. The print shows Tanning as a young art student carrying her portfolio of drawings, which when opened proves to hold her psychic self, her self-projection as an erotic fantasy, naked from the waist down (whatever part of her body that does not show with the portfolio closed). She rides a red-feathered cyclopean humanoid of indeterminate sex which, in turn, rides a white avian creature on a perch; the red creature is wringing the neck of the white creature. Although Tanning shows herself at a very young age -- perhaps still an adolescent -- the monstrous birds suggest she is significantly older. \r\n\r\nShe moved to New York City in 1935 and in the 1940s met the surrealist painter Max Ernst, whom she would marry in 1946. Ernst characteristically personified his iconic motifs as savant bird forms, and thus this young girl riding high over two surrealist (in her own style) birds, in a tableau filled with sex and violence, must evoke her existence after the war, when she and Ernst lived in France for the most part.",NEGATIVE,0.635165
24934,Giacomo Leopardi,"Sandri has imagined the spiritual portrait of Italy's greatest Romantic poet, Giacomo Leopardi (1798-1837), basing his image on lifetime painted portraits of the sickly, melancholic Leopardi. Sandri himself suffered from mental illness: the lithograph is dated to the year preceding his first entrance into an insane asylum. He would be repeatedly hospitalized until his death, in the hospital, in 1937. Sandri's intense image of the author, whose suffering from physical ills and unconsummated loves was the basis for his tragic poetry, must derive some of its affecting introspective mood from the artist's own troubled mind. \r\n\r\nSandri's work has received critical attention in Italy, most recently in exhibitions in Milan (2001,S. Ambrogio Museum, ""From sufferance to faith"") and Rome (2002, Palazzo delle Esposizioni, ""Art, madness, a language of the soul""). It seems as if the emphasis was placed on his drawn portraits of fellow patients, as well as his landscapes, which seem to be greatly appreciated by critics. In 1965, the critic L. Bracchi, on the occasion of a posthumous exhibition in Saronno, wrote, ""I recall above all his graphic work which is impressive. From a psychological point of view I especially recall the splendid drawings of th

In [219]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def scrape_page(url):
    '''
    A function to webscrape primary image off of the Harvard collections 
    website that will work in a Jupyter Notebook.
    To run: await scrape_page(url)
    
    Args
        url(url): The url of the page to scrape

    Returns
        image_url(url): The first image (url) of the page scraped
    '''
    async with async_playwright() as p:
        # Launch a headless browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Navigate to the page
        await page.goto(url)  # Replace with your URL

        # Get the page content
        html_content = await page.content()

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract image URLs
        image_urls = []
        for img_tag in soup.find_all('img'):
            src = img_tag.get('src')
            if src and src.startswith("https://nrs.hvrd.art/urn-3:HUAM"):
                src = src.replace('&amp;', '&')
                image_urls.append(src)
            
            
        return image_urls[0]

        # Close the browser
        await browser.close()


In [246]:
i = commentary_df.loc[24419, 'url']  # Get URL directly

try:
    new_image_url = await scrape_page(i)  # Await the scrape
    commentary_df.loc[24419, 'primaryimageurl'] = new_image_url  # Correct assignment

except Exception as e:
    print(f"Error: {e}")  # Better error handling

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [264]:


for idx in [5993, 24419, 24934, 11545]:

    # URL of the image
    image_url = commentary_df.loc[idx]['primaryimageurl']  #.values[0]
    if pd.notna(image_url) and str(image_url).strip() not in ["", "NaN", "None"]:
        display(Image(url=image_url, width=300))
        print(image_url)
    else:
        print("No valid image URL found.")

https://nrs.harvard.edu/urn-3:HUAM:INV200624_dynmc


https://nrs.hvrd.art/urn-3:HUAM:INV157845_dynmc?width=256&height=256


https://nrs.harvard.edu/urn-3:HUAM:INV044740_dynmc


https://nrs.harvard.edu/urn-3:HUAM:INV177632_dynmc


In [258]:
open_tanning = 'https://www.dorotheatanning.org/images/originals/92.7.03Portfolioopen836.jpg'
display(Image(url=open_tanning, width=300))